In [1]:
''' This code follows the PINN - Navier Stokes project done by Computational Domain, but adapted for our secific scenarios'''
import torch
import torch.nn as nn
import numpy as np
import scipy.io
from matplotlib import pyplot as plt

nu = 0.01
batch_size = 1000  # Define a batch size to control memory usage

# Load data and create training dataset
data = scipy.io.loadmat('cylinder_wake.mat')

U_star = data['U_star']  # N x 2 x T
P_star = data['p_star']  # N x T
t_star = data['t']  # T x 1
X_star = data['X_star']  # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data
XX = np.tile(X_star[:, 0:1], (1, T))  # N x T
YY = np.tile(X_star[:, 1:2], (1, T))  # N x T
TT = np.tile(t_star, (1, N)).T  # N x T

UU = U_star[:, 0, :]  # N x T
VV = U_star[:, 1, :]  # N x T
PP = P_star  # N x T

x = XX.flatten()[:, None]  # NT x 1
y = YY.flatten()[:, None]  # NT x 1
t = torch.tensor(TT.flatten()[:, None], dtype=torch.float32, requires_grad=True)  # Convert to Tensor

u = torch.tensor(UU.flatten()[:, None], dtype=torch.float32)  # Convert to Tensor
v = torch.tensor(VV.flatten()[:, None], dtype=torch.float32)  # Convert to Tensor
p = torch.tensor(PP.flatten()[:, None], dtype=torch.float32)  # Convert to Tensor

# Skip every other time step
skip_factor = 2
t_skipped = t[::skip_factor]  # Skip every other time step
u_skipped = u[::skip_factor]
v_skipped = v[::skip_factor]
p_skipped = p[::skip_factor]

# Note: x and y remain the same, only the time-related data is skipped
x_skipped = torch.tensor(np.tile(X_star[:, 0], T//skip_factor), dtype=torch.float32, requires_grad=True).reshape(-1, 1)
y_skipped = torch.tensor(np.tile(X_star[:, 1], T//skip_factor), dtype=torch.float32, requires_grad=True).reshape(-1, 1)

# Define a simple ResNet block
class ResNetBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ResNetBlock, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        return self.activation(self.linear(x)) + x

# Define the ResNet-style neural network with 7 hidden layers
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.input_layer = nn.Linear(3, 20)
        self.res_block1 = ResNetBlock(20, 20)
        self.res_block2 = ResNetBlock(20, 20)
        self.res_block3 = ResNetBlock(20, 20)
        self.res_block4 = ResNetBlock(20, 20)
        self.res_block5 = ResNetBlock(20, 20)
        self.res_block6 = ResNetBlock(20, 20)
        self.res_block7 = ResNetBlock(20, 20)
        self.output_layer = nn.Linear(20, 3)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        x = self.activation(self.input_layer(x))
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.res_block7(x)
        x = self.output_layer(x)
        return x

net = ResNet()

# Define the loss function
mse = nn.MSELoss()

# function written by Computational Domain
def create_pde(x, y, t):
    res = net(torch.hstack((x, y, t)))
    psi, p = res[:, 0:1], res[:, 1:2]

    u = torch.autograd.grad(psi, y, grad_outputs=torch.ones_like(psi), create_graph=True)[0]  
    v = -1. * torch.autograd.grad(psi, x, grad_outputs=torch.ones_like(psi), create_graph=True)[0]

    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]
    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
    p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

    f = u_t + u * u_x + v * u_y + p_x - nu * (u_xx + u_yy)
    g = v_t + u * v_x + v * v_y + p_y - nu * (v_xx + v_yy)

    return u, v, p, f, g

# Use LBFGS optimizer with early stopping
optimizer_lbfgs = torch.optim.LBFGS(net.parameters(), lr=1, max_iter=20000, max_eval=50000,
                                    history_size=50, tolerance_grad=1e-05, tolerance_change=0.5 * np.finfo(float).eps,
                                    line_search_fn="strong_wolfe")

loss_history = []
f_residual_history = []
g_residual_history = []
lbfgs_iter = 0  # Initialize the LBFGS iteration counter
patience = 200  # Number of iterations with no significant improvement before stopping
min_delta = 1e-4  # Minimum change to qualify as an improvement
best_loss = float('inf')
patience_counter = 0

# Fine-tuning with LBFGS and early stopping in batches
for i in range(0, x_skipped.shape[0], batch_size):
    x_batch = x_skipped[i:i+batch_size].detach().clone().requires_grad_(True)
    y_batch = y_skipped[i:i+batch_size].detach().clone().requires_grad_(True)
    t_batch = t_skipped[i:i+batch_size].detach().clone().requires_grad_(True)
    u_batch = u_skipped[i:i+batch_size]
    v_batch = v_skipped[i:i+batch_size]
    p_batch = p_skipped[i:i+batch_size]
    
    def closure():
        global lbfgs_iter, best_loss, patience_counter  # Access the global iteration variables
        optimizer_lbfgs.zero_grad()

        u_prediction, v_prediction, p_prediction, f_prediction, g_prediction = create_pde(x_batch, y_batch, t_batch)
        u_loss = mse(u_prediction, u_batch)
        v_loss = mse(v_prediction, v_batch)
        p_loss = mse(p_prediction, p_batch)
        f_loss = mse(f_prediction, torch.zeros_like(f_prediction))
        g_loss = mse(g_prediction, torch.zeros_like(g_prediction))

        loss = u_loss + v_loss + p_loss + f_loss + g_loss
        loss.backward()

        # Print the current iteration and loss
        lbfgs_iter += 1
        if lbfgs_iter % 10 == 0:
            print(f'LBFGS Iteration {lbfgs_iter}, Loss: {loss.item()}')

        # Early stopping logic
        if loss.item() < best_loss - min_delta:
            best_loss = loss.item()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                return loss

        # Track loss and residuals
        loss_history.append(loss.item())
        f_residual_history.append(f_loss.item())
        g_residual_history.append(g_loss.item())

        return loss
    
    optimizer_lbfgs.step(closure)

# Plot the loss curve
plt.figure()
plt.plot(loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.show()

# Plot f and g residuals
plt.figure()
plt.plot(f_residual_history, label='f Residual')
plt.plot(g_residual_history, label='g Residual')
plt.xlabel('Iteration')
plt.ylabel('Residual')
plt.title('f and g Residuals Curve')
plt.legend()
plt.show()

# Save the trained model
torch.save(net.state_dict(), 'pinn_skipped_timesteps_resnet_lbfgs.pt')

# Load the trained model and evaluate
net.eval()



LBFGS Iteration 10, Loss: 0.2839852571487427
LBFGS Iteration 20, Loss: 0.023229466751217842


In [ ]:
import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# Assuming necessary data and trained model are available:
# x_train, y_train, t_train, u_train, v_train, p_train, net (the trained model)

# Function to create and save an animation for a specific field on training data
def create_training_animation(field_name, values_fn, vmin, vmax, filename):
    fig, ax = plt.subplots()

    # Initialize the first frame using the first batch
    batch_start = 0
    batch_end = min(batch_size, x_skipped.shape[0])  # Set initial batch
    x_batch = x_skipped[batch_start:batch_end]
    y_batch = y_skipped[batch_start:batch_end]
    t_batch = t_skipped[batch_start:batch_end]

    t_frame = torch.tensor(np.full(x_batch.shape, t_batch[0].item()), dtype=torch.float32, requires_grad=True)
    u_out, v_out, p_out, _, _ = create_pde(x_batch, y_batch, t_frame)
    field_values = values_fn(u_out, v_out, p_out).detach().cpu().numpy().flatten()

    contour = ax.tricontourf(x_batch.detach().numpy().flatten(), y_batch.detach().numpy().flatten(), field_values, levels=20, cmap='jet', vmin=vmin, vmax=vmax)
    colorbar = fig.colorbar(contour, ax=ax, label=field_name)  # Add colorbar once

    ax.set_xlim(x_batch.min().item(), x_batch.max().item())  # Set x-axis limits
    ax.set_ylim(y_batch.min().item(), y_batch.max().item())  # Set y-axis limits
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'{field_name} at t = {t_batch[0][0].item()}')

    def animate(i):
        nonlocal contour, batch_start, batch_end, x_batch, y_batch, t_batch
        for c in contour.collections:
            c.remove()

        # Update batch indices
        batch_start = (i * batch_size) % x_skipped.shape[0]
        batch_end = min(batch_start + batch_size, x_skipped.shape[0])

        x_batch = x_skipped[batch_start:batch_end]
        y_batch = y_skipped[batch_start:batch_end]
        t_batch = t_skipped[batch_start:batch_end]

        t_frame = torch.tensor(np.full(x_batch.shape, t_batch[0].item()), dtype=torch.float32, requires_grad=True)
        u_out, v_out, p_out, _, _ = create_pde(x_batch, y_batch, t_frame)
        field_values = values_fn(u_out, v_out, p_out).detach().cpu().numpy().flatten()

        contour = ax.tricontourf(x_batch.detach().numpy().flatten(), y_batch.detach().numpy().flatten(), field_values, levels=20, cmap='jet', vmin=vmin, vmax=vmax)
        ax.set_title(f'{field_name} at t = {t_batch[0][0].item()}')

    ani = animation.FuncAnimation(fig, animate, frames=min(100, t_skipped.shape[0] // batch_size), interval=200, blit=False)
    ani.save(filename, writer='imagemagick', fps=10)
    plt.close(fig)

# Determine value ranges for proper color scaling using the first batch
t_frame = torch.tensor(np.full(x_skipped[:batch_size].shape, t_skipped[0].item()), dtype=torch.float32, requires_grad=True)
u_out, v_out, p_out, _, _ = create_pde(x_skipped[:batch_size], y_skipped[:batch_size], t_frame)  # Extract only the first three values

vmin_p, vmax_p = torch.min(p_out).item(), torch.max(p_out).item()
vmin_u, vmax_u = torch.min(u_out).item(), torch.max(u_out).item()
vmin_v, vmax_v = torch.min(v_out).item(), torch.max(v_out).item()

# Create and save animations for each field on training data
create_training_animation('Pressure Field', lambda u_out, v_out, p_out: p_out, vmin_p, vmax_p, 'pressure_training_animation.gif')
create_training_animation('U Velocity Field', lambda u_out, v_out, p_out: u_out, vmin_u, vmax_u, 'u_velocity_training_animation.gif')
create_training_animation('V Velocity Field', lambda u_out, v_out, p_out: v_out, vmin_v, vmax_v, 'v_velocity_training_animation.gif')
